# Assignment 4: Using Aggregate and Conditional Queries

## DSCi 504: Intro to SQL

Symphony Hopkins

September 12, 2022

**Part 1: To identify the depth of OPC's MTB sales, it is important to identify summary information within the data. Begin by finding the minimum and maximum order values within the data. Include the following columns in your query:**

- **Order ID**
- **Minimum Order**
- **Maximum Order**

In [26]:
--Finding Min
SELECT o.ord_id, o.order_tot
FROM dsci_504.orders AS o
WHERE o.order_tot = 
(SELECT MIN(o.order_tot)
FROM dsci_504.orders AS o);

--Finding Max
SELECT o.ord_id, o.order_tot
FROM dsci_504.orders AS o
WHERE o.order_tot = 
(SELECT MAX(o.order_tot) 
FROM dsci_504.orders AS o);

(6 row(s) affected)

(2 row(s) affected)

Total execution time: 00:00:00.004

ord_id,order_tot
4455545,1850
2965446,1850
2596420,1850
5571075,1850
2358837,1850
4438686,1850


ord_id,order_tot
1121775,8580
4350229,8580


**Q: What was the minimum order amount?**

```
A: $1850

```

**Q: What was the maximum order amount?**

```
A: $8580

```

**Part 2: Determine the validity of your findings by looking at another numeric field in one of the tables that should match or be close to both of these values. If the values are too far off, we may have a data discrepancy or could possibly have some discounted items. We need to ensure that we can count on the values in the fields. Run a query to find the minimum and maximum cost of mountain bikes in the data.**

In [31]:
--Finding Min
SELECT p.prod_id, p.prod_name, p.prod_price
FROM dsci_504.products AS p
WHERE p.prod_price = 
(SELECT MIN(p.prod_price)
FROM dsci_504.products AS p);

--Finding Max
SELECT p.prod_id, p.prod_name, p.prod_price
FROM dsci_504.products AS p
WHERE p.prod_price = 
(SELECT MAX(p.prod_price)
FROM dsci_504.products AS p);


(1 row(s) affected)

(1 row(s) affected)

Total execution time: 00:00:00.004

prod_id,prod_name,prod_price
63,Air9,1850


prod_id,prod_name,prod_price
14,SB150,8000


**Q: What was the minimum product price?**

```
A: $1850

```

**Q: What is the maximum product price?**

```
A: $8000

```

**Q: Is there a discrepancy in the data? If so, what is assumptions or conclusions can you draw from any discrepancies?**

```
A:  There is a discrepancy between the maximum order total and the maximum product price, but if you consider the different tax rates then this discrepancy makes sense. 

```

**<span style="color: rgb(45, 59, 69); font-family: &quot;Lato Extended&quot;, Lato, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(255, 255, 255);">Part 3: Write a query that identifies the transactions with the biggest difference between the sale price of the bike and the order total. Assume each order is ONLY for a mountain bike and any accessories are extra order numbers</span>.**

**You may need to perform a simple join as discussed in last week's assignment if a query is required. In case you forgot the syntax, it is include below.**

```
SELECT [table 1 col]...[table 2 col]....
FROM [table 1], [table 2]
WHERE [conditions] AND [table 1 id] = [table 2 id];

```

In [71]:
SELECT o.ord_id
      ,o.ord_tax_loc
      ,o.order_tot
      ,p.prod_price
      ,(o.order_tot - p.prod_price) as difference
FROM dsci_504.orders AS o
INNER JOIN dsci_504.products AS p 
ON o.prod_id = p.prod_id
ORDER BY (o.order_tot - p.prod_price) DESC
LIMIT 10;

(10 row(s) affected)

Total execution time: 00:00:00.004

ord_id,ord_tax_loc,order_tot,prod_price,difference
4409334,23,4278.93,2799,1479.93
4350229,4,8580,8000,580
1121775,5,8580,8000,580
1273816,4,8258.24,7699.99,558.25
6666642,5,8258.24,7699.99,558.25
5234755,5,8114.17,7565.66,548.51
5992300,4,8114.17,7565.66,548.51
7328173,12,8238.99,7699.99,539.00
2028513,23,8238.99,7699.99,539.00
1592079,23,8238.99,7699.99,539.00


**Q: What are two unique things that we can glean from this transaction?**

```
A: Two things we saw from this transaction: The biggest difference between sales price and order total was $1479.93; and there is a huge gap between the first order and other orders' differences (the difference is nearly tripled compared to the second order).

```

**Part 4: Create a query to determine which tax zone this order falls into. To make sure you return the correct record, use the order\_track\_num as a condition.**

In [67]:
SELECT o.*
FROM dsci_504.orders AS o
INNER JOIN dsci_504.products AS p 
ON o.prod_id = p.prod_id
WHERE o.ord_track_num = 'PAC5820182-1';

(1 row(s) affected)

Total execution time: 00:00:00.037

ord_id,ord_date,ord_tax_loc,order_tot,ord_ship_add,ord_ship_date,ord_track_num,prod_id,warehouse_id,cus_id
4409334,2019-05-05,23,4278.93,true,2019-05-12,PAC5820182-1,9,2,1


**Q: What is one thing that stands out about this transaction?**

```
A: This order took place is tax location 23, which seems to frequently appear when we look at the top orders with the biggest difference between sale price and order total (see Part 3).

```

**Q: Query the taxes table to identify which state the the ord\_tax\_loc refers to**

In [78]:
--Before update
SELECT q1.ord_track_num
      ,q1.ord_tax_loc
      ,t.tax_location
      ,t.tax_rate
FROM
(SELECT o.*
FROM dsci_504.orders AS o
INNER JOIN dsci_504.products AS p 
ON o.prod_id = p.prod_id
WHERE o.ord_track_num = 'PAC5820182-1') AS q1
INNER JOIN dsci_504.taxes AS t
ON CAST(q1.ord_tax_loc AS SMALLINT) = t.tax_id;

(1 row(s) affected)

Total execution time: 00:00:00.002

ord_track_num,ord_tax_loc,tax_location,tax_rate
PAC5820182-1,23,RI,7


**Part 5: Update the tax rate in the taxes table to reflect the current tax rate of 8.875 for the tax rate location in Part 4.**

In [80]:
UPDATE dsci_504.taxes
SET tax_rate = 8.875
WHERE tax_location = 'RI';

Commands completed successfully

Total execution time: 00:00:00.002

In [81]:
--After update
SELECT q1.ord_track_num
      ,q1.ord_tax_loc
      ,t.tax_location
      ,t.tax_rate
FROM
(SELECT o.*
FROM dsci_504.orders AS o
INNER JOIN dsci_504.products AS p 
ON o.prod_id = p.prod_id
WHERE o.ord_track_num = 'PAC5820182-1') AS q1
INNER JOIN dsci_504.taxes AS t
ON CAST(q1.ord_tax_loc AS SMALLINT) = t.tax_id;

(1 row(s) affected)

Total execution time: 00:00:00.002

ord_track_num,ord_tax_loc,tax_location,tax_rate
PAC5820182-1,23,RI,8.875


**Part 6: Find the average tax rate for all orders. Return the tax location and the sum of all orders for each tax rate location. Group the results by tax location with the highest tax rate listed first. Round the tax rate to the nearest 3 decimals.**

In [16]:
--Average Tax Rate for All Orders
SELECT ROUND(AVG(t.tax_rate),3) AS avg_tax_rate_all_orders
FROM
(SELECT o.*
FROM dsci_504.orders AS o
INNER JOIN dsci_504.products AS p 
ON o.prod_id = p.prod_id) AS q1
INNER JOIN dsci_504.taxes AS t
ON CAST(q1.ord_tax_loc AS SMALLINT) = t.tax_id;

--Sum of All Orders for Each Tax Location 
SELECT t.tax_location
      ,t.tax_rate
      ,SUM(q1.order_tot)
FROM
(SELECT o.*
FROM dsci_504.orders AS o
INNER JOIN dsci_504.products AS p 
ON o.prod_id = p.prod_id) AS q1
INNER JOIN dsci_504.taxes AS t
ON CAST(q1.ord_tax_loc AS SMALLINT) = t.tax_id
GROUP BY t.tax_location, t.tax_rate
ORDER BY t.tax_rate DESC;

(1 row(s) affected)

(30 row(s) affected)

Total execution time: 00:00:00.010

avg_tax_rate_all_orders
5.184


tax_location,tax_rate,sum
MN,8.875,393329.09
RI,8.875,439401.58
KS,8.52,430319.76
CO,7.25,396104.52
CA,7.25,358063.48
IN,7,395199.12
WA,6.5,458163.23
CT,6.35,415228.10
IA,6,354991.17
WV,6,436695.57


**Q: What other tax rates issues are in the result? Research the current state sales tax rate for any states that has a missing or heavily skewed tax rate and write UPDATE queries to correct the data. Because rate can change over time, only change the rates that have invalid values as seen. Others will be fixe din later weeks.**

In [51]:
/*To determine which data points skew the data, we have to find the outliers. To find the outliers we have to find Q1 and Q3, which will also
help us find the IQR. When we find the IQR, we can determine the threshold for the outliers.*/

SELECT p.*
      ,(p.q3 - p.q1) AS IQR
      ,(p.q1 - (1.5 * (p.q3 - p.q1))) AS lower_threshold
      ,(p.q3 + (1.5 * (p.q3 - p.q1))) AS upper_threshold
FROM
(SELECT PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY t.tax_rate) AS Q1
       ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY t.tax_rate) AS Q3
FROM dsci_504.taxes AS t) AS p;

/* From the query above, we found that any tax rate lower than 0.73 or above 9.84 should be updated since those values are
outliers. We can create a query to find the tax rates that are outliers: */

SELECT *
FROM dsci_504.taxes AS t
WHERE t.tax_rate <= 0.73 OR t.tax_rate >= 9.84;

/*Using the results from the query above, we found that these following states are outliers: AK, DE, PA, and OR. To update their values, we used 
tax rates from 2019, since that was the most recent year in the data. DE and OR had 0% tax rates in 2019, so only AK and PA need to be updated. 
Source: https://taxfoundation.org/sales-tax-rates-2019/ */

UPDATE dsci_504.taxes
SET tax_rate = 1.43
WHERE tax_location = 'AK';

UPDATE dsci_504.taxes
SET tax_rate = 6.34
WHERE tax_location = 'PA';

--Because the tax rates have been updates, it will affect the stats values above if this query is ran again.

(1 row(s) affected)

(2 row(s) affected)

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:00.023

q1,q3,iqr,lower_threshold,upper_threshold
4.375,6.425,2.05,1.3000000000000003,9.5


tax_id,tax_location,tax_rate
7,DE,0
21,OR,0


**Part 7: Identity the number of sales that have Alasks as the tax location**

In [43]:
SELECT t.tax_location
      ,COUNT(q1.ord_track_num) as sales_count
FROM
(SELECT o.*
FROM dsci_504.orders AS o
INNER JOIN dsci_504.products AS p 
ON o.prod_id = p.prod_id) AS q1
INNER JOIN dsci_504.taxes AS t
ON CAST(q1.ord_tax_loc AS SMALLINT) = t.tax_id
WHERE t.tax_location = 'AL'
GROUP BY q1.ord_tax_loc, t.tax_location;


(1 row(s) affected)

Total execution time: 00:00:00.002

tax_location,sales_count
AL,73


**Q:  How many rows were affected by the AK sales tax change?**

```
A: 73 Rows

```

**Q: What happened when you tried to query using the ord\_tax\_loc? Fix any issues you identified**

```
A: When you try to query ord_tax_loc using an integer, you get an error because the ord_tax_loc column's data type is character.

```

In [55]:
--Converting ord_tax_loc data type from CHAR(2) to SMALLINT
ALTER TABLE dsci_504.orders
ALTER COLUMN ord_tax_loc TYPE SMALLINT 
USING ord_tax_loc::SMALLINT;

Commands completed successfully

Total execution time: 00:00:00.025

 **Part 8: Write a query to identify the minimum, average, and maximum sales by warehouse. Round all values to the nearest 2 decimals.**

In [59]:
SELECT o.warehouse_id
      ,ROUND(MIN(o.order_tot), 2) AS min_order_total
      ,ROUND(MAX(o.order_tot), 2) AS max_order_total
      ,ROUND(AVG(o.order_tot), 2) AS avg_order_total
FROM dsci_504.orders AS o
GROUP BY o.warehouse_id
ORDER BY o.warehouse_id;

(3 row(s) affected)

Total execution time: 00:00:00.002

warehouse_id,min_order_total,max_order_total,avg_order_total
1,1850.00,8480.00,4824.38
2,1850.00,8580.00,4833.84
3,1850.00,8580.00,4775.24


**Q: Which warehouse had the most order value?**

```
A: Warehouse 2 had the highest average order values.

```

 **Part 9: To determine future employee value based on the number of returned items it may be worth noting how much has been returned over time and which customer has the highest and lowest returns. Write a query to return the customer with the highest number of items returned and the highest return amount. Group the result by the customer's last name. Be sure to round to the nearest 2 decimal places where appropriate.**

In [34]:
SELECT c.cus_last_name
      ,SUM(r.tot_ret_item_cnt) AS tot_ret_item_cnt
      ,SUM(r.tot_ret_amnt) AS tot_ret_amnt
FROM dsci_504.racs AS r
INNER JOIN dsci_504.customers AS c
ON r.cus_id = c.cus_id
GROUP BY c.cus_last_name
ORDER BY SUM(r.tot_ret_item_cnt) DESC, SUM(r.tot_ret_amnt) DESC;


(85 row(s) affected)

Total execution time: 00:00:00.005

cus_last_name,tot_ret_item_cnt,tot_ret_amnt
Jones,154,15702
Smith,145,13823
Jackson,109,8063
Smith-Warren,100,8626
Thomason,100,5547
Calahan,98,8989
Cooper,89,8009
Deleon,87,6077
Peters,85,5638
Conley,83,6698


**Q: How many rows were returned?**

```
A: 85

```

**Q: Explain in your own words what the last query you wrote accomplished. Why are the results the way they are?**

```
A: We grouped the results by the customers' last names, however, this is not an accurate representation of how many items were returned or how much the total return amount for each customer was because we grouped multiple customers with the same last name together.

```

**Q: Amend the above query to return the total amount of returns by customer.**

In [52]:
SELECT c.cus_last_name
      ,c.cus_first_name
      ,SUM(r.tot_ret_item_cnt) AS tot_ret_item_cnt
FROM dsci_504.racs AS r
INNER JOIN dsci_504.customers AS c
ON r.cus_id = c.cus_id
GROUP BY c.cus_last_name, c.cus_first_name
ORDER BY SUM(r.tot_ret_item_cnt) DESC
LIMIT 10;

SELECT c.cus_last_name
      ,c.cus_first_name
      ,SUM(r.tot_ret_amnt) AS tot_ret_amnt
FROM dsci_504.racs AS r
INNER JOIN dsci_504.customers AS c
ON r.cus_id = c.cus_id
GROUP BY c.cus_last_name, c.cus_first_name
ORDER BY SUM(r.tot_ret_amnt) DESC
LIMIT 10;


(10 row(s) affected)

(10 row(s) affected)

Total execution time: 00:00:00.015

cus_last_name,cus_first_name,tot_ret_item_cnt
Thomason,Joshua,31
Jones,Valerie,25
Slain,Abigail,24
Jackson,Eric,23
Applewood,Mads,23
Deleon,Reginald,22
Smith-Warren,Blain,22
Moore,Joshua,22
Smith,Moritz,22
Peters,Margo,21


cus_last_name,cus_first_name,tot_ret_amnt
Jones,Valerie,2723
Jamieson,Adam,2030
Manchester,Chris,2006
Manning,Nadine,1929
Atkins,Wilkie,1845
Banks,Margo,1813
Presley,Erin,1788
Jones,Harper,1730
Jones,Alister,1718
Riochardson,Chris,1717


**Q: Which customer had the most in returns? How much were they?**

```
A: Joshua Thomas had the most total items returned with 31 items. Valerie Jones had the highest total return amount with $2723.

```

In [47]:
SELECT COUNT(q1.tot_ret_item_cnt)
FROM
(SELECT c.cus_last_name
      ,c.cus_first_name
      ,SUM(r.tot_ret_item_cnt) AS tot_ret_item_cnt
FROM dsci_504.racs AS r
INNER JOIN dsci_504.customers AS c
ON r.cus_id = c.cus_id
GROUP BY c.cus_last_name, c.cus_first_name) AS q1
WHERE q1.tot_ret_item_cnt = 0;

(1 row(s) affected)

Total execution time: 00:00:00.005

count
316


**Q: How many customers had zero returns?**

```
A: 316

```

**Part 10: Often times it is benefical to easily bin your customers into segments for further analysis. First, create a new column in the customers table to hold the new data. Name the column "cus\_app\_lvl" for Customer Appreciation Level and make it 10 variable characters in size. Select the range of unique customer appreciation codes then create three bins of codes consisting of "low", "moderate", and "high" appreciation levels. For reference, customer appreciation codes will not exceed 25. To make sure there are no inherent outliers, be sure to widen teh third quartile of your range and keep a widened inner quartile range.**

In [72]:
ALTER TABLE dsci_504.customers
ADD COLUMN cus_app_lvl VARCHAR(10);

/* Not sure if these are the best bins, but I chose these numbers based on the loyalty levels
in Part 11. */

UPDATE dsci_504.customers
SET cus_app_lvl = CASE
WHEN cus_app_cd <= 3 THEN 'high'
WHEN cus_app_cd > 3 AND cus_app_cd <= 7 THEN 'moderate'
WHEN cus_app_cd > 7 AND cus_app_cd <= 25 THEN 'low'
END;

Commands completed successfully

Total execution time: 00:00:00.019

**Test your query by querying the count of all 'Low' customer appreciation customers in the customers table**

In [73]:
SELECT COUNT(c.cus_app_lvl)
FROM dsci_504.customers AS c
WHERE c.cus_app_lvl = 'low';

(1 row(s) affected)

Total execution time: 00:00:00.003

count
14


**Q: How many low appreciation customers are there?**

```
A: 14

```

**Q:  Perform additional queries and determine which customer appreciation level class has the most customers. Output the results as a bar chart to determine the type of skew the data has. Use the following settings:**

- **Data Direction: Vertical**
- **Y Axis Label: Count of Customers**
- **Y Axis Max: 1500**
- **Y Axis Min: 0**
- **X Axis Label: Customer Appreciation Level**
- **Legend Location: Bottom**

In [74]:
SELECT c.cus_app_lvl
      ,COUNT(c.cus_app_lvl) AS cus_app_lvl
FROM dsci_504.customers AS c 
GROUP BY c.cus_app_lvl
ORDER BY c.cus_app_lvl = 'high'
        ,c.cus_app_lvl = 'moderate'
        ,c.cus_app_lvl = 'low';

(3 row(s) affected)

Total execution time: 00:00:00.004

cus_app_lvl,cus_app_lvl
low,14
moderate,1471
high,1134


**Q: Which apprecaition class has the most customers?**

```
A: Low

```

**Part 11: Write a conditional query using a CASE statement to display a column that labels customers as 'Loyal', 'Fence', 'Potential Churn', 'Imminent Churn' or 'Undetermined' based on their customer appreciation code (see list below). Be sure to return appropriate customer demographics with the result.**

1. **3or less = Loyal**
2. **Greater than 3 but less than 7 = Fence**
3. **7 or greater but less than or equal to 10 = Potential Churn**
4. **Greater than 10 = Imminent Churn**
5. **Other = Undetermined**

In [82]:
ALTER TABLE dsci_504.customers
ADD COLUMN cus_loyalty VARCHAR(20);

UPDATE dsci_504.customers
SET cus_loyalty = CASE
WHEN cus_app_cd <= 3 THEN 'Loyal'
WHEN cus_app_cd > 3 AND cus_app_cd < 7 THEN 'Fence'
WHEN cus_app_cd >= 7 AND cus_app_cd <= 10 THEN 'Potential Churn'
WHEN cus_app_cd > 10 THEN 'Imminent Churn'
ELSE 'Undetermined'
END;

SELECT c.cus_loyalty
      ,COUNT(c.cus_loyalty) AS cus_loyalty
FROM dsci_504.customers AS c 
GROUP BY c.cus_loyalty;

SELECT c.cus_id
      ,c.cus_loyalty
FROM dsci_504.customers AS c 
LIMIT 5;


Commands completed successfully

(4 row(s) affected)

(5 row(s) affected)

Total execution time: 00:00:00.028

cus_loyalty,cus_loyalty
Loyal,1134
Fence,1108
Potential Churn,366
Imminent Churn,11


cus_id,cus_loyalty
194,Loyal
730,Potential Churn
731,Fence
732,Fence
733,Loyal


**Q: What is the customer ID of the 5th record returned in the result?**

```
A: 733

```

**Part 12: Write a conditional query that identifies each warehouse as Northeast, South, West, or Non-Regional and compare which warehouses all orders from Ohio were shipped from. Include the customer ID, state, and warehouse region in your output. You may need to llok ahead to the JOIN lessons to complete this query.**

In [122]:
/*The warehouses are in the following areas: 

-Columbus, Ohio : Midwest
-Sacremento, California: West
-Dallas, Texas : South

I am going to assume that the instructions meant Midwest and not Northeast since there are
no warehouses in that region. */

ALTER TABLE dsci_504.warehouses
ADD COLUMN region VARCHAR(15);
UPDATE dsci_504.warehouses
SET region = CASE
WHEN warehouse_id = 1 THEN 'West'
WHEN warehouse_id = 2 THEN 'South'
WHEN warehouse_id = 3 THEN 'Midwest'
ELSE 'Non-Regional'
END;

--All Ohio orders with regional information.
SELECT q2.cus_id
      ,q2.state
      ,w.region
FROM
(SELECT q1.cus_id
      ,s.state
      ,q1.warehouse_id
FROM
(SELECT c.cus_id
       ,c.cus_state
       ,o.warehouse_id
FROM dsci_504.customers AS c 
INNER JOIN dsci_504.orders AS o
ON c.cus_id = o.cus_id) AS q1
INNER JOIN dsci_504.states AS s
ON q1.cus_state = s.state_id
WHERE s.state = 'OH') AS q2
INNER JOIN dsci_504.warehouses AS w
ON w.warehouse_id = q2.warehouse_id;

--Number of Ohio Orders for Each Warehouse Region
SELECT w.region
      ,COUNT(w.region) AS ohio_orders
FROM
(SELECT q1.cus_id
      ,s.state
      ,q1.warehouse_id
FROM
(SELECT c.cus_id
       ,c.cus_state
       ,o.warehouse_id
FROM dsci_504.customers AS c 
INNER JOIN dsci_504.orders AS o
ON c.cus_id = o.cus_id) AS q1
INNER JOIN dsci_504.states AS s
ON q1.cus_state = s.state_id
WHERE s.state = 'OH') AS q2
INNER JOIN dsci_504.warehouses AS w
ON w.warehouse_id = q2.warehouse_id
GROUP BY w.region;


Commands completed successfully

(87 row(s) affected)

(3 row(s) affected)

Total execution time: 00:00:00.010

cus_id,state,region
795,OH,South
851,OH,West
898,OH,Midwest
925,OH,South
1043,OH,Midwest
1048,OH,South
1119,OH,West
1124,OH,West
1159,OH,West
1168,OH,West


region,ohio_orders
Midwest,22
South,29
West,36


**Q: How many orders were shipped to Ohio customers?**

```
A: 87

```

**Q: Summarize what you learned about using aggregates and CASE statements. Explain some insightful facts you were able to identify in your outputs today.**

```
A: I learned that aggregate functions must be used with the group by clause. I also learned that aggregate functions cannot be nested. Regarding CASE statements, I learned that they exist! I did not know this before, but they have become a helpful tool for categorizing values in different columns. Some insightful facts I learned from this assignment were that some companies evaluate their customers based on their appreciation and loyalty, which is usefult if you want to gauge a company's customer retention. 

```